In [3]:
import openai
import json
with open("config.json", 'r') as config_file:
    config = json.load(config_file)

AZURE_OPENAI_API_KEY = config["AZURE_OPENAI_API_KEY"]
AZURE_OPENAI_ENDPOINT = config["AZURE_OPENAI_ENDPOINT"]
DEPLOYMENT_NAME = config["DEPLOYMENT_NAME"]
# Azure OpenAI details

# Create OpenAI client
client = openai.AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,
    api_version="2024-02-01",
    azure_endpoint=AZURE_OPENAI_ENDPOINT
)

# Call the GPT-4o model
response = client.chat.completions.create(
    model=DEPLOYMENT_NAME,  # Deployment name of GPT-4o
    messages=[
        {"role": "user", "content": "What is the capital of France?"}
    ]
)

# Print response
print(response.choices[0].message.content)


The capital of France is Paris.


In [4]:
from langchain.chat_models import AzureChatOpenAI

llm1 = AzureChatOpenAI(
    api_key=AZURE_OPENAI_API_KEY, 
    azure_endpoint=AZURE_OPENAI_ENDPOINT, 
    deployment_name=DEPLOYMENT_NAME, 
    model="gpt-4o",
    api_version="2024-02-01")

response = llm1.invoke("What is the capital of France?")

print(response.content)

/var/folders/t4/9ybby8756ql0sh417vwrbf_58j7sdg/T/ipykernel_26278/3291561183.py:3: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm1 = AzureChatOpenAI(


The capital of France is Paris.


In [36]:
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant"),
    ("human", "{question}")
])

In [42]:
chain = prompt | llm1
response = chain.invoke({"question": "What is the capital of France?"})
print(response.content)

The capital of France is Paris.


In [49]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

memory = ConversationBufferMemory(memory_key="history")

prompt = PromptTemplate(input_variables=["history", "question"], template="Here is our conversation:\n{history}\n User: \n{question}\n AI:")
llm_chain = LLMChain(
    llm=llm1,
    prompt=prompt,
    memory=memory,
    verbose=True)

print(llm_chain.invoke({"question": "What is the capital of France?"})["text"])
print(llm_chain.invoke({"question": "What is its population?"})["text"])
print(llm_chain.invoke({"question": "what is the ratio of men vs woman?"})["text"])



> Entering new LLMChain chain...
Prompt after formatting:
Here is our conversation:

 User: 
What is the capital of France?
 AI:

> Finished chain.
The capital of France is Paris.


> Entering new LLMChain chain...
Prompt after formatting:
Here is our conversation:
Human: What is the capital of France?
AI: The capital of France is Paris.
 User: 
What is its population?
 AI:

> Finished chain.
As of the most recent data, the population of Paris is approximately 2.1 million people. However, the population of the larger Paris metropolitan area, known as Île-de-France, is around 12 million people. Population figures can change, so it's always a good idea to check the latest statistics from a reliable source.


> Entering new LLMChain chain...
Prompt after formatting:
Here is our conversation:
Human: What is the capital of France?
AI: The capital of France is Paris.
Human: What is its population?
AI: As of the most recent data, the population of Paris is approximately 2.1 million people. 